## Gradio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive/Zinnia Gradio"

'Copy of hotel_resnet34.pkl'   hotel_id_code_mapping.csv   submission.csv
 gradio.ipynb		       hotel_resnet34.pkl	  'Test Image.jpg'


In [ ]:
# 1) Point at your Drive folder
DRIVE_FOLDER      = "/content/drive/My Drive/Zinnia Gradio"
WEIGHTS_PATH      = f"{DRIVE_FOLDER}/Copy of hotel_resnet34.pkl"
MAPPING_CSV_PATH  = f"{DRIVE_FOLDER}/hotel_id_code_mapping.csv"



In [ ]:
# 2) Reload everything from Drive
import timm, pandas as pd, numpy as np, cv2
from albumentations import Compose, ToFloat
from albumentations.pytorch import ToTensorV2
from PIL import Image
import torch


In [ ]:
# 2a) load your hotel_id mapping
mapping_df        = pd.read_csv(MAPPING_CSV_PATH)
hotel_id_code_map = mapping_df.set_index("hotel_id_code")["hotel_id"].to_dict()


In [ ]:
# 2b) redefine your model class
class HotelIdModel(torch.nn.Module):
    def __init__(self, n_classes, backbone_name="resnet34"):
        super().__init__()
        self.backbone = timm.create_model(backbone_name,
                                          num_classes=n_classes,
                                          pretrained=False)
    def forward(self, x):
        return self.backbone(x)

In [ ]:
# 2c) instantiate + load weights
device   = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_classes = len(hotel_id_code_map)
model    = HotelIdModel(n_classes, "resnet34")
state    = torch.load(WEIGHTS_PATH, map_location=device)
model.load_state_dict(state)
model.to(device).eval()
print("✅ Model reloaded and ready")


✅ Model reloaded and ready


In [ ]:
# 3) your preprocessing & TTA predict helper (unchanged)
base_transform = Compose([ ToFloat(), ToTensorV2() ])

def pad_and_resize(img, size=256):
    h, w = img.shape[:2]
    diff = abs(h - w)//2
    if h > w:
        img = cv2.copyMakeBorder(img, 0,0, diff,diff, cv2.BORDER_CONSTANT)
    else:
        img = cv2.copyMakeBorder(img, diff,diff,0,0, cv2.BORDER_CONSTANT)
    return cv2.resize(img, (size,size))

def predict_top5(pil_img):
    img    = np.array(pil_img)                # PIL→RGB numpy
    img    = pad_and_resize(img, size=256)
    tensor = base_transform(image=img)["image"].unsqueeze(0).to(device)
    with torch.no_grad():
        out1 = torch.sigmoid(model(tensor))
        out2 = torch.sigmoid(model(torch.flip(tensor, dims=[-1])))
        avg  = (out1 + out2)/2
    idxs = torch.topk(avg, k=5, dim=1).indices.cpu().numpy().ravel()
    return [hotel_id_code_map[int(i)] for i in idxs]

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.3 MB/s eta 0:00:00


In [49]:
import gradio as gr

css = """
.gradio-container { background: white; color: black; }

/* Title styling */
h1 {
  color: white;
  background: #0097a7;
  font-weight: normal;
  font-family: Arial, sans-serif;
  font-size:40px;
  padding: 0.5rem;
  text-align: center;
}

/* Tabs styling */
[role="tablist"] [role="tab"] {
  font-weight: bold !important;
  color: #555 !important;
}
[role="tablist"] [role="tab"][aria-selected="true"] {
  color: #000 !important;
}

/* Upload prompt styling */
#upload-desc, #upload-desc * {
  color: #003366 !important;
  font-weight: bold !important;
  font-size: 1.1rem;
  text-align: center;
  margin-bottom: 0.5rem;
}

/* Info tab text styling */
#welcome-info, #welcome-info * {
  color: #003366 !important;
}
#resnet-info, #resnet-info * {
  color: #003366 !important;
}
#eff-info, #eff-info * {
  color: #003366 !important;
}

/* Center the row contents */
#predict-row {
  display: flex !important;
  align-items: center !important;
  justify-content: center !important;
  gap: 2rem;
  margin-top: 1rem;
}

/* Output box styling */
#output-box {
  background: #e0f7fa;
  border-radius: 8px;
  box-shadow: 0 4px 12px rgba(0,0,0,0.1);
  padding: 1.5rem;
  min-width: 250px;
  max-width: 300px;
}

/* Individual ID item styling */
.id-item {
  font-size: 1.2rem;
  color: #00796b;
  font-weight: bold;
  text-align: center;
  margin: 0.5rem 0;
}
"""

welcome_description = """
## Welcome to the Hotel Identification Demo!

This project supports efforts to combat human trafficking by matching hotel-room images to their source hotels.
We demonstrate two deep-learning approaches:

- **ResNet-34 classification**: Directly predicts hotel IDs in under 30 min.
- **EfficientNet-b0 retrieval**: Embeds images and uses cosine similarity for higher MAP@5, at the cost of longer runtime.

Select a tab to learn more or upload your own image in the Predict tab to see ResNet-34 in action.
"""

resnet_description = """
## ResNet-34 Classification Model

| Configuration      | Run Time    | MAP@5  |
|--------------------|-------------|--------|
| **Baseline**       | 15 minutes  | 0.156  |
| **+ Test-Time Aug**| 30 minutes  | 0.185  |

**Feature Engineering:**
We apply Albumentations transforms on-the-fly—HorizontalFlip, ShiftScaleRotate, OpticalDistortion, Perspective, CoarseDropout, and RandomBrightnessContrast—to simulate occlusions, lighting changes, and viewpoint variations.

**Test-Time Augmentation (TTA):**
We generate three horizontally-flipped variants of each test image and average their sigmoid outputs (with a slight weight on the original). This boosts MAP@5 from 0.156 → 0.185.

> **Takeaway:**
> ResNet-34 with TTA delivers a fast (≤ 30 min) inference pipeline and strong top-5 recall over 3,116 classes—ideal for large-scale hotel identification.
"""


effnet_description = """
## EfficientNet-b0 Embedding Model

| Configuration        | Run Time   | MAP@5  |
|----------------------|------------|--------|
| **EfficientNet-b0**  | 8+ hours   | 0.204  |

**Embedding + Retrieval:**
1. Produce a 512-dim vector for each 256×256 image.
2. L2-normalize and compute cosine similarity against a precomputed bank of embeddings.
3. Return the top-K nearest hotel IDs.

> **Important:**
> We do *not* claim one model is definitively superior—each provides unique insights.
> In urgent real-world settings (e.g. human trafficking investigations), fast turnaround is critical.
> That’s why we demo ResNet-34 here—EfficientNet-b0 is reserved for deeper offline analysis.
"""

with gr.Blocks(theme=gr.themes.Soft(), css=css) as demo:
    gr.Markdown("# 🏨 Hotel Identification", elem_id="title")

    with gr.Tabs():
        # 1) Welcome tab
        with gr.TabItem("Welcome"):
            gr.Markdown(welcome_description, elem_id="welcome-info")

        # 2) ResNet Info tab
        with gr.TabItem("ResNet-34 Info"):
            gr.Markdown(resnet_description, elem_id="resnet-info")

        # 3) Predict tab
        with gr.TabItem("ResNet-34 Prediction Demo"):
            gr.Markdown("Upload Image", elem_id="upload-desc")
            with gr.Row(elem_id="predict-row"):
                with gr.Column(scale=1, min_width=300):
                    img_in = gr.Image(type="pil", label="")
                    btn    = gr.Button("🔍 Predict", variant="primary")
                with gr.Column(scale=1, min_width=300):
                    placeholder = "<div class='id-item' style='color:#aaa;font-style:italic;'>Your top-5 IDs will appear here</div>"
                    out = gr.HTML(placeholder, elem_id="output-box")
            def predict_pretty(pil_img):
                ids = predict_top5(pil_img)
                return "".join(f"<div class='id-item'>• {h}</div>" for h in ids)
            btn.click(fn=predict_pretty, inputs=[img_in], outputs=[out])

        # 4) EfficientNet Info tab
        with gr.TabItem("EfficientNet-b0 Info"):
            gr.Markdown(effnet_description, elem_id="eff-info")

    demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://405effaa4e538afcf5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
